In [4]:
import os
import pandas as pd
import requests
import json

We will read the input excel file that we got  from client. this file has the start and end geocoordinates of the different plants. We have to assess different route between these plants

In [5]:
input_from_client = pd.read_excel(r'Input_from_client.xlsx')

In [6]:
input_from_client.head()
# input_from_client.info()

,uid,start,end,start_lat,start_lon,end_lat,end_lon
0,1,Bidadi,Ambikapur,12.782286,77.424975,23.112297,83.182082
1,2,Bidadi,Bhilai,12.782286,77.424975,21.208241,81.379906
2,3,Bidadi,Raipur,12.782286,77.424975,21.219474,81.631294
3,4,Becharaji Plant,JHANSI,23.495345,72.022300,25.445902,78.589940
4,5,Becharaji Plant,SHAHDOL - JBP,23.495345,72.022300,23.302021,81.369715


Now we will make different Route API call with Driving and Trucking profile to chech how different these 2 profiles are 

In [ ]:
# URL of the Route API call in driving profile
routing_API_Driving_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_adv/driving/'

# Define the parameters for the API request
params = {
    'geometries': 'polyline6',
    'overview': 'full'
}

for index, row in input_from_client.iterrows():

    coordinates = f"{row['start_lon']},{row['start_lat']};{row['end_lon']},{row['end_lat']}?"
    print(coordinates)

    routing_API_response = requests.get(routing_API_Driving_url + coordinates, params=params)

    try:
        routing_API_data = routing_API_response.json()

        input_from_client.loc[index, 'Driving_Duration(s)'] = routing_API_data['routes'][0]['duration']
        input_from_client.loc[index, 'Driving_Distance(m)'] = routing_API_data['routes'][0]['distance']
        # input_from_client.loc[index, 'Driving_Geometry'] = json.dumps(routing_API_data['routes'][0]['geometry'],ensure_ascii=False).replace('"', '')

        filename = f"GeometryCreatedwithRouteAPI/DrivingGeometry/{row['uid']}_drivingRoute.txt"
        with open(filename, 'w', encoding='utf-8') as text_file:
            text_file.write(json.dumps(routing_API_data['routes'][0]['geometry'],ensure_ascii=False).replace('"', ''))

        print(f"The API call for request {index} was successful")
    except:
        print(f"There was an error with the API call for request {index}")


In [ ]:
input_from_client.head(10)

Now the Route API call with Trucking profile

In [ ]:
# URL of the Route API call in driving profile
routing_API_Trucking_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_adv/trucking/'

# Define the parameters for the API request
params = {
    'geometries': 'polyline6',
    'overview': 'full'
}

for index, row in input_from_client.iterrows():

    coordinates = f"{row['start_lon']},{row['start_lat']};{row['end_lon']},{row['end_lat']}?"
    print(coordinates)

    routing_API_response = requests.get(routing_API_Trucking_url + coordinates, params=params)

    try:
        routing_API_data = routing_API_response.json()

        input_from_client.loc[index, 'Trucking_Duration(s)'] = routing_API_data['routes'][0]['duration']
        input_from_client.loc[index, 'Trucking_Distance(m)'] = routing_API_data['routes'][0]['distance']
        # input_from_client.loc[index, 'Driving_Geometry'] = json.dumps(routing_API_data['routes'][0]['geometry'],ensure_ascii=False).replace('"', '')

        filename = f"GeometryCreatedwithRouteAPI/TruckingGeometry/{row['uid']}_TruckingRoute.txt"
        with open(filename, 'w', encoding='utf-8') as text_file:
            text_file.write(json.dumps(routing_API_data['routes'][0]['geometry'],ensure_ascii=False).replace('"', ''))

        print(f"The API call for request {index} was successful")
    except:
        print(f"There was an error with the API call for request {index}")

In [20]:
input_from_client.head(10)

,uid,start,end,start_lat,start_lon,end_lat,end_lon,Driving_Duration(s),Driving_Distance(m),Trucking_Duration(s),Trucking_Distance(m)
0,1,Bidadi,Ambikapur,12.782286,77.424975,23.112297,83.182082,102041.1,1727601.9,129433.6,1738745.0
1,2,Bidadi,Bhilai,12.782286,77.424975,21.208241,81.379906,79418.7,1369632.5,100838.5,1380775.6
2,3,Bidadi,Raipur,12.782286,77.424975,21.219474,81.631294,81441.0,1399348.5,103051.5,1410491.6
3,4,Becharaji Plant,JHANSI,23.495345,72.022300,25.445902,78.589940,49872.9,907597.0,63829.3,904886.6
4,5,Becharaji Plant,SHAHDOL - JBP,23.495345,72.022300,23.302021,81.369715,70920.6,1171071.3,93930.3,1180464.2
5,6,Becharaji Plant,SHAHDOL,23.495345,72.022300,23.302021,81.369715,70920.6,1171071.3,93930.3,1180464.2
6,7,Becharaji Plant,SATNA,23.495345,72.022300,24.572832,80.845080,65047.2,1184206.2,81655.5,1181495.8
7,8,Becharaji Plant,RATLAM,23.495345,72.022300,23.321017,75.027726,27471.0,441971.6,35244.7,456403.0
8,9,Becharaji Plant,NEEMUCH,23.495345,72.022300,24.455081,74.875342,26942.0,436934.2,34128.1,434568.1
9,10,Becharaji Plant,REWA,23.495345,72.022300,24.543591,81.319384,68696.1,1240618.4,85739.9,1237386.5


I have saved all the geomteries from Route API calls in the text files to use them for further analysis. I choose Text files becuase these geometries are too large, some of them have 150k characters. Once the VS code is close the Dataframe is removed, hence it is better to save these  dataframes as a .txt file and then read back into pandas dataframe.

Also written the overview of Distance and Duration on excel file

In [21]:
input_from_client.to_excel('SummaryAndOverview.xlsx')

Now since we have the geometries and overview of Routes we can create a visulization map on QGIS to see the overview of the route, how the Truck Route are different from the regular 4-wheeler routes

In [16]:
import pandas as pd
import folium
from folium import FeatureGroup, GeoJson
from folium import PolyLine
from IPython.display import display

# Assuming your DataFrame is named 'df' with columns 'latitude' and 'longitude'
df = pd.read_excel('SummaryAndOverview.xlsx')  # Replace with your actual file path

# Define function to decode polylines (replace with your actual path)
def decode_polyline(polyline_file):
  with open(polyline_file, 'r') as f:
    encoded_polyline = f.read().strip()
  return folium.PolyLine(locations=polyline.decode(encoded_polyline))

# Create a base map centered on the average latitude and longitude of your data
average_latitude = df['start_lat'].mean()
average_longitude = df['start_lon'].mean()

# Replace with your own tile URL template
tiles = 'https://apis.mappls.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/map_tile/{z}/{x}/{y}.png'

m = folium.Map(location=[average_latitude, average_longitude], zoom_start=5, tiles=tiles, attr='MapmyIndia')

# Create a FeatureGroup to hold markers and polylines
feature_group = FeatureGroup(name="Markers and Polylines")

# Add markers for each geocoordinate in the DataFrame
for index, row in df.iterrows():
    latitude = row['start_lat']
    longitude = row['start_lon']
    popup_text = row['start']  # Customize popup text as needed
    feature_group.add_child(folium.Marker([latitude, longitude], popup=popup_text))

# Read and add polylines from text files (modify as needed)
polyline_files = [f"C:/Users/Mohit/Desktop/Code/Projects/TruckVsDrivingRouteAnalysis/GeometryCreatedwithRouteAPI/DrivingGeometry/1_drivingRoute.txt", f"C:/Users/Mohit/Desktop/Code/Projects/TruckVsDrivingRouteAnalysis/GeometryCreatedwithRouteAPI/DrivingGeometry/2_drivingRoute.txt"]  # Replace with your file paths
for polyline_file in polyline_files:
    polyline = decode_polyline(polyline_file)
    feature_group.add_child(polyline)

# Add the FeatureGroup to the map
m.add_child(feature_group)

# Display the map within the Jupyter Notebook
display(m)

AttributeError: 'PolyLine' object has no attribute 'decode'